In [1]:
import math
import numpy as np
import pandas as pd
from decimal import Decimal
import matplotlib.pyplot as plt

In [2]:
def f(x,y):
    return 418.9829*2 - x*np.sin(np.sqrt(abs(x))) - y*np.sin(np.sqrt(abs(y)))

In [50]:
f.__code__.co_argcount

2

### Estructura cromosómica

In [234]:
for x in '12234':
    print(x)

1
2
2
3
4


In [ ]:
np.binary_repr(142,)

In [235]:
[int(d) for d in np.binary_repr(142)]

[1, 0, 0, 0, 1, 1, 1, 0]

In [191]:
class Gen:
    def __init__(self,dominio,precision):
        """
        Parameters
        ----------
        dominio : tuple
            Límites (liminf, limsup) de la variable en cuestión.
        precision : int
            Número de cifras decimales luego del punto para la variable en cuestión. Debe ser mayor al máximo número de decimales en el dominio
        """       
        self.dominio = dominio
        self.precision = precision
        self.len = self._length()

        # Offset lb -> all(0) ; ub -> all(1)
        self.norm = (Decimal(str(dominio[1])) - Decimal(str(dominio[0]))) *10**precision / int('1'*self.len,2)
        self.offset = Decimal(str(dominio[0])) * 10**precision

    
    def _length(self):
        """Calcula el número de bits en un gen.

        Returns
        -------
        int
            Número de bits del gen <-> variable.
        """
        # Limites
        rango = self.dominio[1] - self.dominio[0]
        cell_count = math.ceil(np.log2(rango))

        # Decimales
        cell_count += math.ceil(np.log2(10**self.precision - 1))
        return cell_count
    
    def translate(self,x,mode='decode'):
        '''
        Traductor para codificar/decodificar (reales <-> binario) genes. 

        Parameters
        ----------
        x : float or str
            Gen a traducir (float si es un real, str si es binario).
        mode : str (code,decode)
            Uno de dos modos: 'code' para pasar de número real a binario, y 'decode' para pasar de binario a real.

        Returns
        -------
        str or float
            Gen traducido (str si x era un real, float si x era binario).
        '''
        
        if mode == 'code':
            ## Real -> Binario
            
            # Cortar decimales por redondeo
            x = float(x) # failsafe
            x = round(x,self.precision)

            # Eliminar punto
            x_int,x_dec = str(x).split(".")
            x_dec = x_dec.ljust(self.precision,"0") # fill precision

            # Representación entera
            x_rep = int(x_int + x_dec)

            # Offset -> ran >= 0
            x_rep = (Decimal(str(x_rep)) - self.offset) / self.norm
            x_rep = round(x_rep) # Nuevos decimales son de orden mayor a la precision
            return np.binary_repr(x_rep, width=self.len)
        
        elif mode == 'decode':
            ## Binario -> Real

            # Convierte a entero base 10
            x = int(x,2)

            # Extraer Offset y Normalizar
            x = x * self.norm + self.offset
            x = str(round(x)) # Nuevos decimales son de orden mayor a la precision

            # Añadir el punto decimal
            int_part = x[:-self.precision]
            dec_part = x[-self.precision:]
            x_float = Decimal(int_part + '.' + dec_part)

            return float(x_float)

In [192]:
class Cromosoma:
    def __init__(self,*genes):
        """
        Parameters
        ----------
        genes : Gen
            Gen(es) del cromosoma ordenado(s)
        """       
        self.genes = genes
        self.crom_len = sum([gen.len for gen in genes])

    def representacion(self,x,mode='decode'):
        '''
        Traductor para codificar/decodificar (reales <-> binario) cromosomas a través de sus genes.

        Parameters
        ----------
        x : list o str
            Expresión del cromosoma (lista si los genes se expresan en números reales, str si el cromosoma es binario)
        mode : str (code,decode)
            Uno de dos modos: 'code' para pasar de número real a binario, y 'decode' para pasar de binario a real.

        Returns
        -------
        str o list
            Cromosoma traducido (elementos tipo str si "genes" era una lista, lista si "genes" era binario).
        '''

        if mode == 'code':
            ## Real -> Binario
            genes_bin = [gen.translate(val, mode=mode) for val,gen in zip(x,self.genes)]
            self.cromosoma = ''.join(genes_bin)
            return self.cromosoma
        
        elif mode == 'decode':
            ## Binario -> Real
            genes_lens = [gen.len for gen in self.genes]
            ind_final = np.cumsum(genes_lens)
            genes_r = [gen.translate(x[ind-lens:ind]) for gen, lens, ind in zip(self.genes, genes_lens, ind_final)]
            return genes_r

### Algoritmo

In [243]:
class SimpleEvolutionary(Cromosoma):

    def optimize(self,f,N):
        self.f = f
        self.N = N
        self.poblacion = self.random_generator(self.N,self.crom_len)
        return self.poblacion
    

    ##############################
    #### GENERACIÓN ALEATORIA ####
    def individuo(self,cromosoma):
        return cromosoma, self.f(*self.representacion(cromosoma))
    
    def random_generator(self,N,size):
        '''
        Rutina de generación aleatoria de individuos

        Parameters
        ----------
        N : int
            Número de individuos en la población.
        size : int
            Longitud del cromosoma característico de los individuos en la población.

        Returns
        -------
        array
            Individuos generados como un array de cromosomas (col0) con su aptitud (col1) respectiva.
        '''
        # Creación de población de individuos
        poblacion = np.random.binomial(n=1, p=0.5, size=(N,size)).astype(str)
        poblacion = pd.DataFrame([self.individuo(''.join(cromosoma)) for cromosoma in poblacion],
                                 columns = ['cromosoma','apt']) 

        return poblacion
    

    ###############################
    #### SELECCIÓN: POR RULETA ####

    def ruleta(self):
        '''
        Selección de candidatos a reproducción de la población actual.

        Returns
        -------
        DataFrame
            Individuos seleccionados para reproducirse con la misma dimensión que la población.
        '''
        # Cálculo de proporción de aptitud y límites de probabilidad acumulada
        pi = self.poblacion['apt'] / self.poblacion['apt'].sum()
        pop_lims =  pd.DataFrame({'liminf': (pi).cumsum() - pi,
                                  'limsup': (pi).cumsum()})
        
        # Selección aleatoria de candidatos (lambda = |población|)
        R = np.random.uniform(0,1,pop_lims.shape[0])
        k_pop = pd.DataFrame(map(lambda x: self.poblacion[(pop_lims['liminf'] < x) & (x <= pop_lims['limsup'])].iloc[0], R))
        
        return k_pop
    
    ########################################
    #### OPERADOR CRUZA: CRUZA UN PUNTO ####
    def cruza(self,par1,par2,cr=0.8):
        fail = np.random.uniform(0,1)

        if fail < cr:
            crossover_point = np.random.randint(0, len(par1)) 
            ch1 = par1[:crossover_point] + par2[crossover_point:]
            ch2 = par2[:crossover_point] + par1[crossover_point:]
        else:
            ch1 = par1
            ch2 = par2
        return ch1, ch2

    def mutacion(self,cromosoma,mr=None):
        if not mr:
            mr = 1 / len(cromosoma)
        flip = np.random.binomial(1,mr,len(cromosoma))
        mutated_cr = [str((int(d) + f) % 2) for d,f in zip(cromosoma,flip)]
        mutated_cr = ''.join(mutated_cr)
        return mutated_cr

In [242]:
(0+1)%2

1

In [223]:
if not 1:
    print(1)

### Test

In [194]:
# arbitrary number of genes
Gen1 = Gen((-500,500),5)
Gen2 = Gen((-500,500),3)
CromA = Cromosoma(Gen1,Gen2)

In [244]:
se = SimpleEvolutionary(Gen1,Gen2)

In [245]:
se.optimize(f,10)

,cromosoma,apt
0,11100011000100001111101111001011011000101100111,478.041287
1,01000000000101100010001001001100101110111100110,750.784913
2,00111100100001011010110000110011000111110101110,748.062972
3,10001000010001000111000110111010100000011100100,1075.005690
4,11101001100011100101010010000000000010010010000,259.799895
5,01001111101100011111001101111110010011111101010,677.791329
6,01100111000001000101110000111011010000100010111,827.649336
7,01100110010101010000111111000101000010101101111,666.544516
8,01111011101101000111101001010010101010010010110,799.221935
9,01101001010101010000111010011000000011011000101,878.286891


In [246]:
pars = se.ruleta()
pars

,cromosoma,apt
0,11100011000100001111101111001011011000101100111,478.041287
2,00111100100001011010110000110011000111110101110,748.062972
2,00111100100001011010110000110011000111110101110,748.062972
8,01111011101101000111101001010010101010010010110,799.221935
1,01000000000101100010001001001100101110111100110,750.784913
3,10001000010001000111000110111010100000011100100,1075.005690
2,00111100100001011010110000110011000111110101110,748.062972
9,01101001010101010000111010011000000011011000101,878.286891
7,01100110010101010000111111000101000010101101111,666.544516
8,01111011101101000111101001010010101010010010110,799.221935


In [255]:
ch1,ch2 = se.cruza(pars['cromosoma'].iloc[0],pars['cromosoma'].iloc[1])
ch1

'11100100100001011010110000110011000111110101110'

In [265]:
se.mutacion(ch1)

'11100100100001011010110000110011000111110111110'

In [105]:
N=10
size=CromA.crom_len
poblacion = np.random.binomial(n=1, p=0.5, size=(N,size)).astype(str)
poblacion = np.array([''.join(individuo) for individuo in poblacion]) 

# Cálculo de aptitud
apts = np.array([f(*CromA.representacion(individuo,mode='decode')) for individuo in poblacion])

np.array(list(zip(poblacion, apts)))[:,0]

array(['10111010010010010000100011011001111111110111110',
       '00101000001111111001101011000011110001100110110',
       '01100011000000101110011011011110101100101110001',
       '01010011010111000100000010000111010000101011110',
       '11010001011011110001110111111000111000110011011',
       '01010100000000110010100111111011100000100010110',
       '00010111000110100110111001101000100110111111010',
       '11011000100111100110100110110101101001100101111',
       '00010011000001000001111001111001011000100100111',
       '10100110000011111111110010001011001100011010010'], dtype='<U47')

In [123]:
N=10
size=CromA.crom_len
poblacion = np.random.binomial(n=1, p=0.5, size=(N,size)).astype(str)
poblacion = np.array([''.join(individuo) for individuo in poblacion]) 

# Cálculo de aptitud
apts = np.array([f(*CromA.representacion(individuo,mode='decode')) for individuo in poblacion])

poblacion, apts

(array(['01110100100110000101100000101001011010100001001',
        '10100010001011011001111111110010011101101010011',
        '01101010001111000100001101000110000110111000000',
        '00111000010011101111000100100101101100000011100',
        '10100100001100111000000110101101000000111011110',
        '10101101100010010000011010100001011001100001110',
        '10010111011110110111011101011011000000101011101',
        '11110001110100111000001001101100111011010100000',
        '10001010001001110010010010101000010100010001111',
        '11100101000110000100010101101000100001100001100'], dtype='<U47'),
 array([1056.30115125,  904.51695339,  560.21578389,  346.24981809,
         904.66261779,  983.01487654,  953.61809909,  453.15982296,
         888.11799139,  596.83460652]))

In [141]:
individuos = SimpleEvolutionary(Gen1,Gen2).optimize(f,10)
individuos

,cromosoma,apt
0,10001011010001100010000100111001010011001000011,1105.547877
1,10100110110011110110110110110101100110110111001,767.071722
2,11011000110100000111111000101111010011001001011,892.958987
3,01111100111011100101000111000101110010100001000,568.013314
4,10111100000000111111100000110111100011010010110,667.317125
5,00001111011011100001100000011000110101010011010,1430.265392
6,00100000000111011110101000110001100000011010100,992.977110
7,10100010100110101101111010000011010101000000111,1290.590836
8,11101011011010010011110111010010001001101110111,355.979157
9,01110011011001111000110011110101100010100011011,774.326757


In [162]:
R = np.random.uniform(0,1,individuos['apt'].shape[0])

In [160]:
individuos[(300 < individuos['%']) & (individuos['%'] <= 700)]

,cromosoma,apt
3,01111100111011100101000111000101110010100001000,568.013314
4,10111100000000111111100000110111100011010010110,667.317125
8,11101011011010010011110111010010001001101110111,355.979157


In [163]:
lims = pd.DataFrame({'%':(individuos['apt'] / individuos['apt'].sum()),
              'lf': (individuos['apt'] / individuos['apt'].sum()).cumsum() - (individuos['apt'] / individuos['apt'].sum()),
              'ls': (individuos['apt'] / individuos['apt'].sum()).cumsum()})
lims

,%,lf,ls
0,0.124991,0.000000,0.124991
1,0.086723,0.124991,0.211714
2,0.100956,0.211714,0.312670
3,0.064218,0.312670,0.376888
4,0.075445,0.376888,0.452333
5,0.161702,0.452333,0.614036
6,0.112264,0.614036,0.726299
7,0.145911,0.726299,0.872210
8,0.040246,0.872210,0.912456
9,0.087544,0.912456,1.000000


In [183]:
pd.DataFrame(map(lambda x: individuos[(lims['lf'] < x) & (x <= lims['ls'])].iloc[0], R))

,cromosoma,apt,%
4,10111100000000111111100000110111100011010010110,667.317125,0.075445
2,11011000110100000111111000101111010011001001011,892.958987,0.100956
2,11011000110100000111111000101111010011001001011,892.958987,0.100956
8,11101011011010010011110111010010001001101110111,355.979157,0.040246
7,10100010100110101101111010000011010101000000111,1290.590836,0.145911
0,10001011010001100010000100111001010011001000011,1105.547877,0.124991
2,11011000110100000111111000101111010011001001011,892.958987,0.100956
7,10100010100110101101111010000011010101000000111,1290.590836,0.145911
2,11011000110100000111111000101111010011001001011,892.958987,0.100956
1,10100110110011110110110110110101100110110111001,767.071722,0.086723


In [171]:
individuos[(lims['lf'] < 0.23970917) & (0.23970917 <= lims['ls'])]

,cromosoma,apt,%
2,11011000110100000111111000101111010011001001011,892.958987,0.100956


In [19]:
N = 30
crom_len = CromA.len
pp = np.random.binomial(n=1, p=0.5, size=(N,crom_len))

pp = pp.astype(str)
pp_bin = np.array([''.join(xi) for xi in pp])

In [20]:
_realrep = np.array([CromA.representacion(cr,mode='decode') for cr in pp_bin])
__binrep = np.array([CromA.representacion(cr,mode='code') for cr in _realrep])

for i,j in zip(pp_bin,__binrep):
    if i!=j:
        print('i: ', CromA.representacion(i,mode='decode'), i)
        print('j: ', CromA.representacion(j,mode='decode'), j)
        print('#####################')

i:  [-418.765, 239.40144] 00010100110010111101101111010100100101101001100
j:  [-418.765, 239.40144] 00010100110010111101101111010100100101101001101
#####################
i:  [4.724, -249.64008] 10000001001101011001010000000001011110010110100
j:  [4.724, -249.64008] 10000001001101011001010000000001011110010110011
#####################
i:  [-33.529, -250.79232] 01110111011010101010001111111100110000010011001
j:  [-33.529, -250.79232] 01110111011010101010001111111100110000010011000
#####################
i:  [-456.008, -324.6111] 00001011010000110001001011001110011001001001100
j:  [-456.008, -324.6111] 00001011010000110001001011001110011001001001011
#####################
i:  [-395.206, 270.26561] 00011010110100111100110001010011000000100000100
j:  [-395.206, 270.26561] 00011010110100111100110001010011000000100000011
#####################
i:  [169.29, 141.10996] 10101011010101101001101001000001111111001000001
j:  [169.29, 141.10996] 10101011010101101001101001000001111111001000010
##########

In [21]:
_realrep
   

array([[-418.765  ,  239.40144],
       [   4.724  , -249.64008],
       [ 322.98   ,  225.7134 ],
       [ 413.189  ,  -83.59369],
       [ -90.992  , -401.50003],
       [ -33.529  , -250.79232],
       [ 211.15   ,  369.5504 ],
       [-418.128  ,  441.36653],
       [ 465.819  ,  367.98943],
       [-452.341  , -148.88363],
       [ 416.194  ,  213.1971 ],
       [-300.21   ,  -30.52776],
       [-210.928  , -197.70191],
       [-456.008  , -324.6111 ],
       [-251.8    , -357.38771],
       [ 107.522  , -172.14284],
       [-425.829  ,  -43.60397],
       [-395.206  ,  270.26561],
       [ 169.29   ,  141.10996],
       [-404.475  ,   77.68261],
       [-168.25   ,  303.28398],
       [  -5.466  , -481.4669 ],
       [-149.489  ,  243.40278],
       [-279.72   , -124.2217 ],
       [ 341.711  , -369.22052],
       [-382.11   , -399.54816],
       [-225.209  ,  410.94005],
       [  33.177  , -250.26148],
       [-163.711  ,  213.25166],
       [  86.27   ,  187.95249]])